In [2]:
import lightning as L
from lightning.pytorch.tuner import Tuner

import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import models
from torchvision.transforms import v2

# Data Processing

In [4]:
transform = v2.Compose([
    v2.Resize(244),
    v2.RandomHorizontalFlip(),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

In [5]:
# Download Data from open datasets.
data = {
    split: datasets.CIFAR10(
        root='../dataset/',
        download=True,
        transform=transform,
    ) for split in ['train', 'val']
}

Files already downloaded and verified
Files already downloaded and verified


# Model

In [13]:
class DenseNet121CIFAR10(L.LightningModule):
    def __init__(self, batch_size=1):
        super(DenseNet121CIFAR10, self).__init__()
        self.batch_size = batch_size
        densenet121 = models.densenet121()
        densenet121.classifier = nn.Linear(
            densenet121.classifier.in_features, 10
        )
        self.model = densenet121

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=1e-3)

    def train_dataloader(self):
        return DataLoader(dataset=data['train'], batch_size=self.batch_size, num_workers=4)

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('train_loss', loss, on_epoch=True)
        return loss

    def val_dataloader(self):
        return DataLoader(dataset=data['val'], batch_size=self.batch_size, num_workers=4)

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('val_loss', loss, on_epoch=True)

In [14]:
model = DenseNet121CIFAR10()

# Training 

In [ ]:
trainer = L.Trainer(
    devices='devices',
    accelerator='auto',
    log_every_n_steps=8
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [17]:
if trainer.num_devices == 1:
    tuner = Tuner(trainer)
    tuner.scale_batch_size(model, mode="power")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_steps=3` reached.
Batch size 2 succeeded, trying batch size 4
`Trainer.fit` stopped: `max_steps=3` reached.
Batch size 4 succeeded, trying batch size 8
`Trainer.fit` stopped: `max_steps=3` reached.
Batch size 8 succeeded, trying batch size 16
Batch size 16 failed, trying batch size 8
Finished batch size finder, will continue with full run using batch size 8
Restoring states from the checkpoint path at /home/ceduardosq/Code/2-Projects/1-ForecastingPoverty/poverty_forecasting_peru/.scale_batch_size_b85942b2-d381-449a-baed-2cbe4268ca8e.ckpt
Restored all states from the checkpoint at /home/ceduardosq/Code/2-Projects/1-ForecastingPoverty/poverty_forecasting_peru/.scale_batch_size_b85942b2-d381-449a-baed-2cbe4268ca8e.ckpt


In [ ]:
trainer.fit(model)